In [71]:
import torch
import torch.nn as nn
import random
import numpy as np
import Tokens as tk
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
VOCAB_SIZE = 1201

разбивка текста и создание словаря

In [72]:
import os

samples = []
for sample in os.listdir('samples'):
    with open("samples/" + sample, encoding="utf-8") as text:
        samples.append(text.read().lower())


class QAPair:
    def __init__(self, question, answer):
        self.question = question
        self.answer = answer

dataset = []

for sample in samples:
    lines = sample.splitlines()
    questions = lines[::2]
    answers = lines[1::2]
    for q, a in zip(questions, answers):
        dataset.append(QAPair(q, a))


In [73]:
tdg = tk.TokenDictionaryGenerator(vocabulary_size = VOCAB_SIZE-1)
tokens = tdg.generate_tokens(samples)
tokenizer = tk.Tokenizer(tokens)
tokens.save("tokens.json")

создание модели 
предложение -> hidden
последние слово + hidden -> слово(1)...слово(n)

In [74]:
class RnnTextGen(nn.Module):

    def __init__(self,input_size,inp_lstm_size,hid_size,n_layers,out_size,dropout=0.2) -> None:
        super(RnnTextGen,self).__init__()
        self.input_size = input_size
        self.out_size = out_size
        self.n_layers = n_layers
        self.hidden_size=hid_size
        self.Encoder = nn.Embedding(input_size,inp_lstm_size)
        self.lstm = nn.LSTM(inp_lstm_size,hid_size,n_layers)
        self.dropout = nn.Dropout(dropout)
        self.l1 = nn.Linear(hid_size,out_size)
        self.l2 = nn.Linear(inp_lstm_size,out_size)
        self.attention = nn.MultiheadAttention(out_size,1)
        
    def forward(self,x,key,value,hidden=None):
        x = self.Encoder(x)
        p = self.l2(x)
        aw,_ = self.attention(p.view(-1,self.out_size),key,value)#a - attn output b - attn_wheights)
        x,hidden = self.lstm(x)
        x = self.dropout(x)
        x = self.l1(x)
        x = x[-1]*aw[-1]
        return x,hidden
    
    def init_hidden(self,batch_size=1):
        return (torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True).to(device),
               torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True).to(device))

In [75]:
model=RnnTextGen(VOCAB_SIZE-1,1000,500,2,VOCAB_SIZE).to(device)
key =torch.Tensor(VOCAB_SIZE).to(device).view(-1,VOCAB_SIZE)
value = torch.Tensor(VOCAB_SIZE).to(device).view(-1,VOCAB_SIZE)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, amsgrad=True)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    patience=5, 
    verbose=True, 
    factor=0.5
)

In [76]:
def evaluate(model:RnnTextGen,text:str,prediction_lim:int=15):
    text_idx = torch.LongTensor(list(tokenizer.tokenize(text))).to(device)
    hidden = model.init_hidden()
    inp = text_idx
    predicted_text=""
    for i in range(prediction_lim):
        next_w , hidden = model(inp.to(device),key,value,hidden)
        inp = torch.cat([inp,next_w.argmax().view(-1)])
        if next_w.argmax() == torch.LongTensor([VOCAB_SIZE-1]).to(device):
            break
        word = tokens.decode(int(next_w.argmax()))
        predicted_text +=word
    return predicted_text

In [77]:
def get_batch(dataset:list):
    for qa in dataset:
        question_idx = list(tokenizer.tokenize(qa.question))
        target = list(tokenizer.tokenize(qa.answer))+[tokens.count()]
        test = question_idx+target[:-1]

        target =torch.LongTensor(target).to(device)
        test = torch.LongTensor(test).to(device)
        yield target,test

In [78]:
def train(epoches:int,model:RnnTextGen,batch_size:int)->None:
    """epoches - number of epoches through all dataset
    model - model required to teach
    batch_size - n/a"""

    loss_avg =[]
    for epoch in range(epoches):
        for target,train in get_batch(dataset):
            model.train()

            hidden = model.init_hidden(batch_size)

            output,hidden = model(train,key,value,hidden)
            loss = criterion(output,target)

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            loss_avg.append(loss.item())
            if len(loss_avg) >= 50:
                mean_loss = np.mean(loss_avg)
                print(f'Loss: {mean_loss}')
                scheduler.step(mean_loss)
                model.eval()
                question = random.choice(dataset).question
                answer = evaluate(model,question)
                print(f"Question: {question} \n Answer: {answer}")
                loss_avg = []

обучение модели

In [79]:
train(30, model, 1)

RuntimeError: Expected floating point type for target with class probabilities, got Long

In [80]:
quest = input().lower()
evaluate(model,quest,35)

'удовлетворенностиудовлетворенностиудовлетворенностиудовлетворенностиудовлетворенностиудовлетворенностиудовлетворенностиудовлетворенностиудовлетворенностиудовлетворенностиудовлетворенностиудовлетворенностиудовлетворенностиудовлетворенностиудовлетворенностиудовлетворенностиудовлетворенностиудовлетворенностиудовлетворенностиудовлетворенностиудовлетворенностиудовлетворенностиудовлетворенностиудовлетворенностиудовлетворенностиудовлетворенностиудовлетворенностиудовлетворенностиудовлетворенностиудовлетворенностиудовлетворенностиудовлетворенностиудовлетворенностиудовлетворенностиудовлетворенности'

In [ ]:
torch.save(model,"data.pkl")

In [ ]:
model = torch.load("data.pkl").to(device)

In [ ]:
import torch 
import torch.nn as nn

class abracadabra(nn.Module):
    def __init__(self,size):
        super(abracadabra,self).__init__()
        self.size = size

        self.embeding = nn.Embedding(size,10)
        self.l1 = nn.Linear(10,size)
        self.attention = nn.MultiheadAttention(size,1)
        self.lstm = nn.LSTM(size,10,2)
        self.l2 = nn.Linear(10,size)
        
        
    def forward(self,x,k,v,hidden=None):
        x = self.embeding(x)
        x = self.l1(x)
        a,b = self.attention(x.view(-1,self.size),k.view(-1,self.size),v.view(-1,self.size))#a - attn output b - attn_wheights
        out,hidden = self.lstm(x,hidden)
        out = self.l2(out)
        sumt = a[-1]*out[-1]
        return sumt
    
model = abracadabra(VOCAB_SIZE).to(device)
x=torch.LongTensor([1,2,3,4,5]).to(device)
model(x,key,value)

tensor([ 3.4077e-04, -3.7418e-03,  3.2053e-03,  ...,  1.3814e-03,
         6.1889e-05,  3.8469e-04], device='cuda:0', grad_fn=<MulBackward0>)